## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [32]:
lines = list(open('./news_train.txt', 'r', encoding='utf-8'))

In [38]:
import random
import re
random.shuffle(lines)

In [34]:
with open('./news_test.txt', 'w', encoding='utf-8') as f:
    for line in lines[12000:15000]:
        f.write(line)

In [35]:
from collections import Counter

Counter([line.split('\t')[0] for line in lines[:12000]])

Counter({'style': 242,
         'forces': 987,
         'media': 1655,
         'science': 1737,
         'sport': 1775,
         'culture': 1642,
         'life': 1612,
         'travel': 225,
         'economics': 1675,
         'business': 450})

In [36]:
lines_train = lines[:12000]
lines_test = lines[12000:]

In [39]:
dict_train = {'labels': [], 'headings': [], 'texts': []}
dict_test = {'labels': [], 'headings': [], 'texts': []}

for line in lines_train:
    label, heading, text = line.split('\t')
    dict_train['labels'].append(label)
    dict_train['headings'].append(re.findall(r'\b\w+\b', heading.lower()))
    dict_train['texts'].append(re.findall(r'\b\w+\b', text.lower()))

for line in lines_test:
    label, heading, text = line.split('\t')
    dict_test['labels'].append(label)
    dict_test['headings'].append(re.findall(r'\b\w+\b', heading.lower()))
    dict_test['texts'].append(re.findall(r'\b\w+\b', text.lower()))

In [47]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

dict_train['texts'] = [[morph.parse(word)[0].normal_form for word in text] for text in dict_train['texts']]

In [49]:
dict_test['texts'] = [[morph.parse(word)[0].normal_form for word in text] for text in dict_test['texts']]

In [50]:
from gensim.models import Word2Vec

In [52]:
model = Word2Vec(sentences=dict_train['texts'], workers=4)

In [54]:
model.wv.most_similar(positive=['зима'])

[('зимой', 0.7459695935249329),
 ('столетие', 0.7232862710952759),
 ('планетоид', 0.7136454582214355),
 ('опять', 0.7049204707145691),
 ('лето', 0.6933284997940063),
 ('переноситься', 0.6916874051094055),
 ('датироваться', 0.6900960206985474),
 ('весна', 0.6880436539649963),
 ('отремонтировать', 0.677342414855957),
 ('универсиада', 0.6722579002380371)]

In [55]:
model.wv.most_similar(positive=['спорт'])

[('экспресс', 0.8114804029464722),
 ('р', 0.7826417684555054),
 ('мутко', 0.7095458507537842),
 ('футбол', 0.7056471109390259),
 ('виталий', 0.6898400783538818),
 ('хоккей', 0.6459627151489258),
 ('marca', 0.6164757013320923),
 ('тасс', 0.6100749969482422),
 ('исполком', 0.6044817566871643),
 ('федерация', 0.603024423122406)]

In [112]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import TruncatedSVD
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [134]:
X_train = [' '.join(text) for text in dict_train['texts']]
y_train = dict_train['labels']
X_test = [' '.join(text) for text in dict_test['texts']]
y_test = dict_test['labels']

In [136]:
tfidf = TfidfVectorizer()

X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

In [139]:
RandCV_logreg = RandomizedSearchCV(LogisticRegression(), param_distributions={'C': np.arange(0.01, 1.01, 0.01)})
RandCV_naivebayes = RandomizedSearchCV(MultinomialNB(), param_distributions={'alpha': np.arange(0.5, 1.51, 0.01)})

In [140]:
RandCV_logreg.fit(X_train_vec, y_train)

RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='warn', n_jobs=None,
                                                penalty='l2', random_state=None,
                                                solver='warn', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distr...
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0

In [141]:
y_pred_lreg = RandCV_logreg.best_estimator_.predict(X_test_vec)
print('Precision: ', precision_score(y_pred_lreg, y_test, average='weighted'))
print('Recall: ', recall_score(y_pred_lreg, y_test, average='weighted'))
print('F1 score: ', f1_score(y_pred_lreg, y_test, average='weighted'))
print('Accuracy: ', accuracy_score(y_pred_lreg, y_test))

Precision:  0.8928819108748056
Recall:  0.8673333333333333
F1 score:  0.8759470587331706
Accuracy:  0.8673333333333333


In [142]:
RandCV_naivebayes.fit(X_train_vec, y_train)

RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                           fit_prior=True),
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'alpha': array([0.5 , 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 ,
       0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71,
       0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8 , 0....
       0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.  , 1.01, 1.02, 1.03, 1.04,
       1.05, 1.06, 1.07, 1.08, 1.09, 1.1 , 1.11, 1.12, 1.13, 1.14, 1.15,
       1.16, 1.17, 1.18, 1.19, 1.2 , 1.21, 1.22, 1.23, 1.24, 1.25, 1.26,
       1.27, 1.28, 1.29, 1.3 , 1.31, 1.32, 1.33, 1.34, 1.35, 1.36, 1.37,
       1.38, 1.39, 1.4 , 1.41, 1.42, 1.43, 1.44, 1.45, 1.46, 1.47, 1.48,
       1.49, 1.5 ])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_

In [144]:
y_pred_nb = RandCV_naivebayes.predict(X_test_vec)
print('Precision: ', precision_score(y_pred_nb, y_test, average='weighted'))
print('Recall: ', recall_score(y_pred_nb, y_test, average='weighted'))
print('F1 score: ', f1_score(y_pred_nb, y_test, average='weighted'))
print('Accuracy: ', accuracy_score(y_pred_nb, y_test))

Precision:  0.8809479630495028
Recall:  0.809
F1 score:  0.8382154076508355
Accuracy:  0.809
